In [1]:
import torch
import h5py
import json

In [2]:
representation_files = "/usr/users/johnmwu/contextual-corr-analysis/slurm/repr_files1"
output_file = "/data/sls/temp/johnmwu/contextual-corr-analysis/results3"

In [3]:
with open(representation_files) as f:
    representation_fname_l = [line.strip() for line in f]

In [4]:
# Set `means_d`, `stdevs_d`
# {fname: tensor}
means_d = {}
stdevs_d = {}

### for fname in ... loop

In [6]:
# arbitrary loop variable
# fname = representation_fname_l[0] # bert large. Takes up 13GiB of memory. No. 
fname = representation_fname_l[3] # elmo original

In [7]:
# Create `activations_h5`, `sentence_d`, `indices`
activations_h5 = h5py.File(fname, 'r')
sentence_d = json.loads(activations_h5['sentence_to_index'][0])
temp = {} 
for k, v in sentence_d.items():
    temp[v] = k
sentence_d = temp # {str ix, sentence}
indices = list(sentence_d.keys())

In [8]:
# Dimension of activations tensor
# 2 if 1 layer, 3 if multiple
dim = len(activations_h5['0'].shape)
dim

3

In [11]:
# Set `activations` tensor
activations = torch.cat([torch.tensor(activations_h5[str_ix])
                            for str_ix in sentence_d],
                        dim=0 if dim==2 else 1)

In [14]:
if dim == 2:
    means_d[fname] = torch.mean(activations, dim=0, keepdim=False)
    stdevs_d[fname] = torch.std(activations, dim=0, keepdim=False, unbiased=False)
elif dim == 3:
    means_d[fname] = torch.mean(activations, dim=1, keepdim=False)
    stdevs_d[fname] = torch.std(activations, dim=1, keepdim=False, unbiased=False)

In [ ]:
# full
for fname in tqdm(representation_fname_l, desc='mu, sigma'):
    # Create `activations_h5`, `sentence_d`, `indices`
    activations_h5 = h5py.File(fname, 'r')
    sentence_d = json.loads(activations_h5['sentence_to_index'][0])
    temp = {} 
    for k, v in sentence_d.items():
        temp[v] = k
    sentence_d = temp # {str ix, sentence}
    indices = list(sentence_d.keys())

    # Set `activations` tensor, `dim`
    dim = len(activations_h5['0'].shape)
    activations = torch.cat([torch.tensor(activations_h5[str_ix])
                                for str_ix in sentence_d],
                            dim=0 if dim==2 else 1)

    if dim == 2: 
        means_d[fname] = torch.mean(activations, dim=0, keepdim=False)
        stdevs_d[fname] = torch.std(activations, dim=0, keepdim=False, unbiased=False)
    elif dim == 3:
        means_d[fname] = torch.mean(activations, dim=1, keepdim=False)
        stdevs_d[fname] = torch.std(activations, dim=1, keepdim=False, unbiased=False)